In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

from glow import lenses, time_domain, freq_domain, tools

## Lenses

### Axisymmetric lenses: SIS

In [ ]:
Psi = lenses.Psi_SIS()

Every object has a `display_info()` method

In [ ]:
Psi.display_info()

We can use a couple of simple functions in `tools.py` to plot the lens

In [ ]:
y = 0.3
fig, ax = tools.plot_potential(y, Psi, levels=10)

Also in polar coordinates

In [ ]:
y = 0.3
fig, ax = tools.plot_potential_polar(y, Psi, levels=10)

In [ ]:
y = 0.3
fig, ax = tools.plot_potential_polar2(y, Psi, levels=10)

### Non-axisymmetric lenses: off-center SIS

As a simple example of a non-axisymmetric lens we have the SIS displaced from the center

In [ ]:
p_phys = {'xc1' : 0.1, 'xc2' : 0.2}

Psi = lenses.Psi_offcenterSIS(p_phys)

Psi.display_info()

In [ ]:
y = 0.3
fig, ax = tools.plot_potential(y, Psi, levels=10)

### Combined lenses

Finally, we combine different lenses

In [ ]:
Psi1 = lenses.Psi_offcenterSIS({'psi0' : 0.5, 'xc1' : 0.,  'xc2' : 0.})
Psi2 = lenses.Psi_offcenterSIS({'psi0' : 0.5, 'xc1' : 0.1, 'xc2' : 0.2})

Psi = lenses.CombinedLens({'lenses' : [Psi1, Psi2]})

Psi.display_info()

In [ ]:
y = 0.3
fig, ax = tools.plot_potential(y, Psi, levels=25)

Right now, only the combination of non-axisymmetric lenses is implemented. For axisymmetric lenses, use the off-center version at 0.

## Time domain

### Single contour

For WL, the single contour is now also implemented in Python. However, keep in mind that it **does not** check if we are indeed in the WL regime. It is also quite slow. 

In [ ]:
Psi = lenses.Psi_SIS()
It = time_domain.It_SingleContour(Psi, y=2.)

fig, ax = plt.subplots()
ax.plot(It.t_grid, It.It_grid/2/np.pi)
ax.set_xlabel(r'$\tau$')
ax.set_ylabel(r'$I(\tau)/2\pi$')
ax.set_xscale('log');

It includes a method to obtain the contour contributing to a given time delay

In [ ]:
tau1 = 0.3
tau2 = 1.3

x1_1, x2_1 = It.get_contour(tau1)
x1_2, x2_2 = It.get_contour(tau2)

fig, ax = plt.subplots()
ax.plot(x1_1, x2_1, label='$\\tau = %g$' % tau1)
ax.plot(x1_2, x2_2, label='$\\tau = %g$' % tau2)
ax.set_xlabel('$x_1$')
ax.set_ylabel('$x_2$')
ax.legend(loc=1);

### Analytic SIS

The analytic integral for the SIS is also implemented now

In [ ]:
y = 0.3
It_exact = time_domain.It_AnalyticSIS(y, p_prec={'tmax':2.5})

fig, ax = plt.subplots()
ax.plot(It_exact.t_grid, It_exact.It_grid/2/np.pi)
ax.set_xlabel(r'$\tau$')
ax.set_ylabel(r'$I(\tau)/2\pi$');

### Single integral

This method can only be applied to axisymmetric lenses but it works for strong lensing.

In [ ]:
y = 0.3

Psi = lenses.Psi_CIS()
It  = time_domain.It_SingleIntegral(Psi, y, {'tmax':2.4, 'sampling':'linear'})

fig, ax = plt.subplots()
ax.plot(It.t_grid, It.It_grid/2./np.pi)
ax.set_xlabel(r'$\tau$')
ax.set_ylabel(r'$I(\tau)/2\pi$');

As with the single contour method, we can also get the contours:

In [ ]:
taus = [0.2, 0.5, 0.6, 0.8, 1.3]
ctrs = [It.get_contour(tau) for tau in taus]

# ----------------------------

fig, ax = plt.subplots()

for i, ctr in enumerate(ctrs):
    label = label='$\\tau = %g$' % taus[i]
    for c in ctr:
        x1, x2 = c
        ax.plot(x1, x2, label=label, c='C%d' % i)
        label = ''
        
ax.set_xlabel('$x_1$')
ax.set_ylabel('$x_2$')
ax.grid(alpha=0.7)
ax.legend();

### Naive grid integration

A very simple implementation of the grid integration is also included (although it is not very efficient). Example for strong lensing with a CIS:

In [ ]:
Psi = lenses.Psi_CIS()
It = time_domain.It_NaiveAreaIntegral(Psi, y=0.3)
It.display_info()

In [ ]:
ts, Its = It.t_grid, It.It_grid
    
fig, ax = plt.subplots()
ax.plot(ts, Its)
ax.set_xlim([0, 3])
ax.set_xlabel("$\\tau$")
ax.set_ylabel("$I(\\tau)$")
plt.tight_layout();

## Frequency domain

### Default settings

In [ ]:
It = time_domain.It_AnalyticSIS(y=2)
Fw  = freq_domain.Fw_FFT_OldReg(It)

The multigrid method can be compared with the standard one (demanding on memory and time)

In [ ]:
ws = Fw.w_grid

fig, ax = plt.subplots()
ax.plot(ws, np.abs(Fw.Fw_grid), label='multigrid')
ax.legend(loc='best')
ax.set_xscale('log')
ax.set_yscale('linear')
ax.set_xlim([ws[0], ws[-1]])
ax.set_xlabel("$w$")
ax.set_ylabel("$|F(w)|$")
plt.tight_layout();